In [507]:
import numpy as np
from numpy import linalg
from sympy import *

In [508]:
# n = 4
n = 2
m = 10
# m = 20
# m = 11 for kowalik-osborne function

In [509]:
# x = np.array([symbols('x{0}'.format(i + 1)) for i in range(n)])
# u = np.array([symbols('u{0}'.format(i + 1)) for i in range(m)])
# y = np.array([symbols('y{0}'.format(i + 1)) for i in range(m)])

In [510]:
# def create_kowalik_osborne_function(x, u, y):
#     return np.array([simplify('{y_i} - ({x1}*({u_i}*{u_i} + {u_i}*{x2})) / ({u_i}*{u_i} + {u_i}*{x3} + {x4})'.format())])

In [511]:
# Brown - Dennis function
def create_brown_denis_function(x, m):
    arr = []
    for i in range(m):
        t = symbols('t{0}'.format(i + 1))
        t_i = t.subs({t: (i + 1) / 5})
        arr.append(simplify('({x_1} + {t_i}*{x_2} - exp({t_i}))^2 + ({x_3} + {x_4}*sin({t_i}) - cos({t_i}))^2'
        .format(x_1 = x[0], x_2 = x[1], x_3 = x[2], x_4 = x[3], t_i = t_i)))
    
    return np.array(arr)

In [512]:
# Jenrich - Sampson function
def CreateJenrichSampsonFunction(x, m):
    arr = []
    for i in range(m):
        # t = symbols('t{0}'.format(i + 1))
        # t_i = t.subs({t: (i + 1) / 5})
        arr.append(simplify('2 + 2*{i} - exp({i}*{x_1}) - exp({i}*{x_2})'.format(x_1 = x[0], x_2 = x[1], i = i + 1)))
    
    return np.array(arr)

In [513]:
x = np.array([symbols('x{0}'.format(i + 1)) for i in range(n)])
# F = create_brown_denis_function(x, m)
# x0 = [25, 5, -5, -1]
# f_exact = 85822.2
F = CreateJenrichSampsonFunction(x, m)
print(F)
x0 = np.array([0.3, 0.4], dtype = 'float')
f_exact = 124.362

[-exp(x1) - exp(x2) + 4 -exp(2*x1) - exp(2*x2) + 6
 -exp(3*x1) - exp(3*x2) + 8 -exp(4*x1) - exp(4*x2) + 10
 -exp(5*x1) - exp(5*x2) + 12 -exp(6*x1) - exp(6*x2) + 14
 -exp(7*x1) - exp(7*x2) + 16 -exp(8*x1) - exp(8*x2) + 18
 -exp(9*x1) - exp(9*x2) + 20 -exp(10*x1) - exp(10*x2) + 22]


In [514]:
# x = np.array([symbols('x{0}'.format(i + 1)) for i in range(4)])
# F = np.array([simplify('{x_1}*exp()')])

In [515]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')

In [516]:
def F_Derivative(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [517]:
def DividedDifferenceOfOperator(x1, x2):
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')

In [518]:
eps = 10 ** -7

In [519]:
def GaussNewtonLeastSquaresMethod(x0, x, F):
    I = 0

    operatorsMultiplication = np.dot(F_Derivative(x0).transpose(), F_Derivative(x0))
    # print(operatorsMultiplication)

    multiplicationOfOperatorAndFunction = np.dot(F_Derivative(x0).transpose(), F_0(x0))

    x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)
    
    print(np.linalg.solve(operatorsMultiplication, multiplicationOfOperatorAndFunction))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I +=1
    
    # while True:
    while I < 12:
        x0 = x1

        operatorsMultiplication = np.dot(F_Derivative(x0).transpose(), F_Derivative(x0))
        multiplicationOfOperatorAndFunction = np.dot(F_Derivative(x0).transpose(), F_0(x0))

        x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)

        if (max(multiplicationOfOperatorAndFunction) < eps):
            break
        
        print(np.linalg.solve(operatorsMultiplication, multiplicationOfOperatorAndFunction))
        print("{0} Ітерація: Значення F = {1} у точці x = {2}\n\n".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [520]:
def PotraLeastSquaresMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)
    
    I = 0

    A  = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
    operatorsMultiplication = np.dot(A.transpose(), A)
    multiplicationOfOperatorAndFunction = np.dot(A.transpose(), F_0(x0))

    x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        A  = (DividedDifferenceOfOperator(x0, x_1) + DividedDifferenceOfOperator(x_2, x0)) - DividedDifferenceOfOperator(x_2, x_1)
        operatorsMultiplication = np.dot(A.transpose(), A)
        multiplicationOfOperatorAndFunction = np.dot(A.transpose(), F_0(x0))

        x1 = x0 - np.dot(np.linalg.inv(operatorsMultiplication), multiplicationOfOperatorAndFunction)

        if (max(np.abs(x1 - x0)) < eps and max(multiplicationOfOperatorAndFunction) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [521]:
# def LeastSquareMethod(x0):
#     print('\n\n')
#     print(np.linalg.norm(F_0(x0)))
#     mult = F_0(x0) * F

In [522]:
print("Метод Гаусса - Ньютона (найменших квадратів)\n\n")
approx_values = GaussNewtonLeastSquaresMethod(x0, x, F)
print(approx_values)
# print(np.linalg.norm(F_0(approx_values)))

Метод Гаусса - Ньютона (найменших квадратів)


[-0.21871557  0.17938005]
1 Ітерація: Значення F = [   1.07328194    1.62340793    1.32108653   -0.38033522   -4.39104136
  -12.22989556  -26.43576379  -51.25786524  -93.81470182 -166.03994824] у точці x = [0.51871557 0.22061995]
[0.09054318 0.06485959]
2 Ітерація: Значення F = [  1.29700329   2.27996135   2.79142709   2.59155767   1.31437333
  -1.59931145  -7.0047437  -16.21099106 -31.22293581 -55.11251008] у точці x = [0.42817239 0.15576036]


[ 0.10419929 -0.3983384 ]
3 Ітерація: Значення F = [ 8.77018113e-01  1.05949536e+00  8.55861833e-02 -2.82845398e+00
 -9.01892674e+00 -2.07731881e+01 -4.20191263e+01 -7.95195013e+01
 -1.44942788e+02 -2.58456492e+02] у точці x = [0.32397309 0.55409876]


[0.22535702 0.07945537]
4 Ітерація: Значення F = [  1.28891664   2.19810711   2.5023219    1.84004664  -0.36921266
  -5.05788255 -13.72402965 -28.77487528 -54.07896704 -95.85375289] у точці x = [0.09861607 0.47464339]


[-0.37251596  0.09336421]
5 Іт